In [1]:
import sqlite3
import pandas as pd
import matplotlib as mpl
import csv

In [20]:
# establish database connection
conn = sqlite3.connect('pursell.db')

# create cursor to execute sql
click = conn.cursor()

# Establish Customer Table


In [3]:
# delete table if already present / establish fresh, clean start to file when working in Kernal > 'Restart & Run All' order
click.execute('''DROP TABLE customer''')

# create customer table
click.execute('''CREATE TABLE customer (
    customer_id INTEGER PRIMARY KEY,
    first VARCHAR(25) NOT NULL,
    last VARCHAR(25) NOT NULL,
    email VARCHAR(75) NOT NULL,
    street VARCHAR(25) NOT NULL,
    city VARCHAR(50) NOT NULL,
    state VARCHAR(15) NOT NULL,
    zip_code INTEGER NOT NULL,
    birthdate DATE NOT NULL,
    payment REAL NOT NULL
)''')

# I know I want to access the csv file and hold that value here in python
# customer_sheet = open('customer.csv')

# I also know python has to read the file that I open
# customer_contents = csv.reader(customer_sheet)



# A 'with-block' means the file automatically closes
with open('customer.csv', 'r') as f:
    # customer_sheet will hold the value of the csv being parsed
    # reader throws an error
    # Dictreader provides a fieldname parameter I can iterate over
    customer_sheet = csv.DictReader(f)
    
    # list comprehension time! 
    
    # to iterate over my csv and store each row and data point respectively
    # by corresponding header fields
    data_customer = [(i['customer_id'], i['first'], i['last'], i['email'], i['street'], i['city'], i['state'], i['zip_code'], i['birthdate'], i['payment']) for i in customer_sheet]


    

# Insert Data for Customer Table

In [4]:
# as for the database
# INSERT INTO customer table the column names and corresponding values per column
customer_insertion = "INSERT INTO customer (customer_id, first, last, email, street, city, state, zip_code, birthdate, payment) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"


In [5]:
# for sql to insert all data points parsed from the csv
click.executemany(customer_insertion, data_customer)

conn.commit() #necessary to save changes to the customer table

# Establish Delivery Table with Data

In [6]:
# delete table if already present
click.execute('''DROP TABLE IF EXISTS delivery''')

# create customer table
click.execute('''CREATE TABLE delivery (
    package TEXT PRIMARY KEY,
    customer_id INTEGER,
    ordered TIMESTAMP,
    delivered TIMESTAMP,
    payment VARCHAR(25),
    sold REAL,
    first TEXT NOT NULL,
    last TEXT NOT NULL,
    street VARCHAR(75) NOT NULL,
    city TEXT,
    state TEXT, 
    zip_code INTEGER,
    purse INTEGER
)''')

# Automatically closing the file means I save a line of code later on
with open('delivery.csv', 'r') as file:
    delivery_sheet = csv.DictReader(file)
    data_delivery = [(i['package'], i['customer_id'], i['ordered'], i['delivered'], i['payment'], i['sold'], i['first'], i['last'], i['street'], i['city'], i['state'], i['zip_code'], i['purse']) for i in delivery_sheet]


# feeding the values python just read, held into data_delivery 
delivery_insertion = "INSERT INTO delivery (package, customer_id, ordered, delivered, payment, sold, first, last, street, city, state, zip_code, purse) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"

# for sql to execute the statement to insert all data points parsed from the csv
click.executemany(delivery_insertion, data_delivery)

conn.commit() #necessary to save changes within the database to the delivery table

# About the Data

Some fun nuance exists and assumptions are to be made. 

**1. Any and all items priced below  250.00 must be assumed to be purse hardware.** 
    
    - Only if the item sold for  250.00 or more is the item definitively a purse because it is impossible to 
    buy a customizable purse for less than the standard box price of 250.00. 
    - Prices include taxes and shipping respectively


**2. We can assume the possibilty exists within the data for the customer address fields to not be the same as the delivery address fields.**
    
    - Two assumptions available from this include 1) gifts and 2) fraud.
        - Likewise, may not be assumed that 'delivery customer first' will always match 'customer first'.


**3. The 'purse' column contains only unique values; the assumption exists that each order is an individual, one-of-a-kind shipment that cannot be reproduced.** A small dataset and lightweight database can still scream "Mission Critical" as I take on the world of fine art and fashion with an entrepreneurial spirit.
    
    - I also took the luxury of nixing the possibility of NULL values. Any NaN value should be impossible given the minimal collection of data to perform business. 
    - Fun statement to leave room for a version 2 in the future featuring expansion of database by adding extra data
        
       

# Time to Query

This is where I get to know my customers a little bit. As a small business, I personally discuss each order with each customer payment, and it's important I'm able to ask questions to make decisions. 

I must know that all my data has been sent to my database, though!



In [7]:
# I need a query to ask how many records exist in my customer table
customer_cells = ('''SELECT COUNT(*)
    FROM customer''')

# a variable to hold the query execution
customer_rows = click.execute(customer_cells).fetchall()

for c in customer_rows:
    print(c)

(100,)


In [8]:
# I want a better view than that. Let's clean that up!

# Let's use some string formatting again
fmt = "{0}"
for cr in customer_rows:
    cr = int(fmt.format(*cr)) #integer casting for easy adding of the variable elsewhere
    print(cr)
    

100


So I know all 100 rows of customer data made it to my database. Let me just double-check my delivery sheet as well by adding my delivery sheet count. I know the database is perfect if it has exactly 200 rows available. 

In [9]:
for d in click.execute('''SELECT COUNT(*) FROM delivery''').fetchall():
    delivery_cells = int(fmt.format(*d)) # integer casting for easy adding in the next line
    print(delivery_cells+cr)

200


In [10]:
##  #create a SELECT statement to interact with database

#select_all = "SELECT * FROM customer"
select_all = "SELECT * FROM delivery ORDER BY ordered" 

##  #retrieve the rows from the database and hold that value for python
rows = click.execute(select_all).fetchall() 



##  #print the rows in python for us to see


## #create an email list
#fmt_email_customer = "{0}) {1} {2}: {3}"  ## #removed sensitive data beyond scope of necessity


## # view a delivery inventory of sorts
fmt_whole_delivery = "{0}--{1}  {2} || {3}    $ {5} ---> {6} {7} ({10})" ## #removed address/sensitive details

for v in rows:
#    customer_email_view = fmt_whole_customer.format(*v)
    delivery_whole_view = fmt_whole_delivery.format(*v)
    print() #extra space for clear view
    print(delivery_whole_view)
    



P94--94  2020-01-13 2:09:16 || 2020-01-23 10:56:34    $ 132.34 ---> Scottie Alliband (Louisiana)

P65--65  2020-02-08 11:34:58 || 2020-02-15 1:18:33    $ 2993.22 ---> Merna Shakesby (Minnesota)

P89--89  2020-02-15 9:17:56 || 2020-02-27 15:54:02    $ 106.53 ---> Celie Di Pietro (Louisiana)

P36--36  2020-02-24 18:08:34 || 2020-03-04 4:52:10    $ 2771.22 ---> Kylen Handrahan (Georgia)

P60--60  2020-03-06 13:27:22 || 2020-03-11 20:35:01    $ 1748.35 ---> Claretta Lumbly (Michigan)

P56--56  2020-03-22 22:45:21 || 2020-08-11 7:46:38    $ 649.81 ---> Aloysius Bulmer (Michigan)

P61--61  2020-03-25 19:54:17 || 2020-04-05 23:10:13    $ 2413.32 ---> Florry Yeowell (Georgia)

P87--87  2020-04-27 22:47:06 || 2020-05-05 14:28:52    $ 1373.74 ---> Hana Borless (Michigan)

P78--78  2020-05-24 22:17:18 || 2020-06-02 16:25:10    $ 2965.18 ---> Juliane Birtle (California)

P40--40  2020-06-03 4:19:21 || 2020-06-16 8:14:59    $ 63.65 ---> Pierson Ingleby (California)

P77--77  2020-06-19 20:08:13 ||

Perfect! So I know that all 200 cells of data points from each of my csv files have successfully transferred into my database, with 100 records per table. 

For my first order of business, I want to send a personalized follow-up gift thanking each customer for their purchase. I decide my preferred method of going about this task is to send a birthday gift to everybody.  

In [27]:
# sql query
birthday_list = ('''SELECT first, birthdate
                FROM customer
                ORDER BY birthdate''')

# view query result
fmt_two = "{0},  {1}"
for r in click.execute(birthday_list).fetchall():
    birthday_list = fmt_two.format(*r)
    # print()           ##extra space in case easier to view
    print(birthday_list)


Penni,  1930-01-03
Raymund,  1930-10-02
Adelle,  1932-01-24
Hugues,  1933-09-09
Chevy,  1935-01-28
Umberto,  1935-04-10
Hana,  1935-11-19
Alisander,  1936-09-18
Harbert,  1937-08-21
Karissa,  1938-10-04
Pierson,  1939-12-20
Clayton,  1940-01-26
Doy,  1940-05-18
Claretta,  1941-02-22
Licha,  1941-05-28
Tersina,  1941-10-27
Rustin,  1942-06-18
Harriott,  1943-03-31
Gaile,  1944-07-12
Fowler,  1945-01-31
Ginger,  1946-08-24
Fonz,  1946-11-05
Marin,  1948-06-20
Darryl,  1949-03-26
Clemence,  1949-09-15
Garrot,  1950-03-11
Matilde,  1950-04-10
Branden,  1951-11-27
Perrine,  1952-11-22
Jehu,  1953-05-08
Christean,  1953-10-18
Aloysius,  1955-06-26
Matias,  1957-04-16
Celie,  1957-06-11
Kara,  1958-12-05
Chryste,  1959-12-15
Tadeas,  1961-04-12
Kara-lynn,  1962-01-09
Jody,  1962-10-04
Kare,  1962-12-23
Rivkah,  1963-05-05
Grannie,  1966-02-02
Kayle,  1966-06-20
Vittorio,  1966-09-21
Cayla,  1967-12-19
Thomasine,  1968-08-25
Therese,  1968-10-08
Juliane,  1968-10-29
Rea,  1968-11-05
Doralin,  

#### I have two types of gifts available:

1. decorated wallets
2. gold-plated tins

To help me split the population between these two options, I choose to make an assumption of which to base my condition upon.

I make an assumption that the earliest generations would likely enjoy the style of wallet available, because the images styled on them resemble more modern cartoon characters from recent pop culture. 

The more reserved appearance of the minimalistic gold tins may then appeal to the older generations. 


In [16]:
# Pretending it's September now, I realize I need to filter by month as well. 
# I'm going to start now, looking at October. 

# sql query
query_october_tin = ('''SELECT first, birthdate
                FROM customer
                WHERE (birthdate LIKE '%-10-%') AND (birthdate <= '1994-12-31')
                ORDER BY birthdate''')

# view query result
for oct_tin in click.execute(query_october_tin).fetchall():
    october_tins = fmt_two.format(*oct_tin)
    print(october_tins)

Raymund, 1930-10-02
Karissa, 1938-10-04
Tersina, 1941-10-27
Christean, 1953-10-18
Jody, 1962-10-04
Therese, 1968-10-08
Juliane, 1968-10-29
Aaren, 1984-10-12
Goddart, 1993-10-12


In [17]:
# Now I'm going to determine the october wallets to prepare: 

# sql query
query_oct_wallets = ('''SELECT first, birthdate
                FROM customer
                WHERE (birthdate LIKE '%-10-%') AND (birthdate >= '1995-01-01')
                ORDER BY birthdate''')

# view query result
for oct_wallet in click.execute(query_oct_wallets).fetchall():
    october_wallets_go_to = fmt_two.format(*oct_wallet)
    print(october_wallets_go_to)

Merna, 1998-10-19


Wow. Only one wallet to send out currently. 

# Now to create an email list to keep in touch with my beloved customers

In [48]:
##  #create a SELECT statement to interact with database

select_all = "SELECT * FROM customer WHERE (birthdate LIKE '%-10-%') ORDER BY birthdate"


##  #retrieve the rows from the database and hold that value for python
rows = click.execute(select_all).fetchall() 



##  #print the rows in python for us to see


## #create an email list
fmt_email_customer = "{1} {2} ({8}) ---> {3}"  ## #removed sensitive data beyond scope of necessity

for v in rows:
    customer_email_view = fmt_email_customer.format(*v)
    print() #extra space for clear view
    print(customer_email_view)
    



Raymund Postlethwaite (1930-10-02) ---> rpostlethwaite2@nasa.gov

Karissa Wannes (1938-10-04) ---> kwannes22@sciencedaily.com

Tersina Youd (1941-10-27) ---> tyoud1v@joomla.org

Christean Sanchez (1953-10-18) ---> csanchez1d@g.co

Jody Willatts (1962-10-04) ---> jwillatts0@wordpress.com

Therese Christmas (1968-10-08) ---> tchristmas4@1und1.de

Juliane Birtle (1968-10-29) ---> jbirtle25@cnn.com

Aaren Kemm (1984-10-12) ---> akemmr@patch.com

Goddart Root (1993-10-12) ---> grootc@columbia.edu

Merna Shakesby (1998-10-19) ---> mshakesby1s@flavors.me


### Now I want to merge and join tables around to get some deeper insight into the data. Maybe find a way to export a document here -- the database as a sql file?

## do pandas explorations /// make in-notebook graphs using matplotlib highlighting sales / questions

## get familiar with tableau / 

In [49]:
conn.close()